In [1]:
import matplotlib.pyplot as plt
import time

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import numpy as np
from itertools import combinations

In [3]:
from pkg.tf_transform import *
from pkg.tf_robot import *
from pkg.constraint import *
from pkg.info import *
from pkg.tf_utils import *
from pkg.rotation_utils import *
from pkg.utils import *
from pkg.ur10 import *
from pkg.geometry import *
from pkg.collision import *
from pkg.distance import *

# Load urdf 

In [4]:
from urdf_parser_py.urdf import URDF
from pkg.ur10 import URDF_PATH, JOINT_NAMES, LINK_NAMES, ZERO_JOINT_POSE
from pkg.joint_utils import *
urdf_content = URDF.from_xml_file(URDF_PATH)

In [5]:
link_names = LINK_NAMES
base_name = LINK_NAMES[0]
joint_names = JOINT_NAMES
link_info_list = get_link_info_list(link_names, urdf_content)

# Visualize

In [6]:
from pkg.ros_rviz import *
pub, joints, rate = get_publisher(JOINT_NAMES)

gitem_list, gframe_dict= get_link_items_offsets(color=(0,1,0,0.5), display=True)
gitem_list += [
    GeometryItem(name='box1', gtype=GeoType.BOX, dims=[0.1,0.1,0.1], color=(0,1,0,1), display=True, collision=True)
]
gframe_dict.update({"box1":GeometryFrame(SE3(Rot_zyx(0,0,0),(0.5,0,0)), "world")
                   })
marker_list = set_markers(gitem_list, gframe_dict, urdf_content)
show_motion([np.array([0]*6)], marker_list, 
            [[gframe_dict[gitem.name] for gitem in gitem_list]], 
            pub, joints, error_skip=1e-6, period=1e-6)

Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
publication OK - base_capsule
publication OK - shoulder_capsule
publication OK - upper_arm_capsule
publication OK - forearm_capsule
publication OK - wrist_1_capsule
publication OK - wrist_2_capsule
publication OK - wrist_3_capsule
publication OK - tool_mesh
publication OK - box1


# Constraint

In [7]:
def get_adjacent_links(link_name, urdf_content):
    adjacent_links = [link_name]
    for k, v in urdf_content.joint_map.items():
        if v.parent == link_name:
            adjacent_links += [v.child]
        if v.child == link_name:
            adjacent_links += [v.parent]
    return list(set(adjacent_links))

In [8]:
class ObjectLayer(layers.Layer):
    def __init__(self, gitem, N_sim, *args, **kwargs):
        self.gitem, self.N_sim = gitem, N_sim
        super(ObjectLayer, self).__init__(*args, **kwargs)
        
    def set_frame(self, Toff_list, link_idx_list, N_link):
        self.Toff_list = tf.constant(Toff_list) # (N_sim, 4,4)
        self.link_one_hot = tf.reshape(tf.one_hot(link_idx_list, N_link), (N_sim,N_link,1,1)) # (N_sim, N_link)
        
    # 변수를 만듭니다.
    def build(self, input_shape):
        pass
    
    @tf.function
    def get_vertice(self):
        return self.vertice

    # call 메서드가 그래프 모드에서 사용되면
    # training 변수는 텐서가 됩니다.
    @tf.function
    def call(self, input=None):
        T_all = input # (N_sim, N_link, 4,4)
        T_act = K.sum(T_all*self.link_one_hot, axis=1) # (N_sim, 4,4)
        T_bo = tf.matmul(T_act, self.Toff_list) # (N_sim, 4,4)
        return T_bo

# Distance

In [9]:
class DistanceCalculator:
    def __init__(self, graph):
        self.graph = graph
        self.N_sim = graph.N_sim
        self.prepare_collision_dat()
            
    def prepare_collision_dat(self):
        for col_case in self.graph.pair_cases:
            setattr(self, "vtx1_"+col_case, self.graph.col_vtx1_dict[col_case])
            setattr(self, "vtx2_"+col_case, self.graph.col_vtx2_dict[col_case])
            setattr(self, "dist1_"+col_case, self.graph.col_dist1_dict[col_case])
            setattr(self, "dist2_"+col_case, self.graph.col_dist2_dict[col_case])
            setattr(self, "zeros_"+col_case, tf.constant(get_zero_points(self.N_sim, self.graph.N_col_dict[col_case]), dtype=tf.float32))
            setattr(self, "N_col_"+col_case, self.graph.N_col_dict[col_case])
            setattr(self, "flag_default_"+col_case, 
                    tf.constant(get_flag_default(self.N_sim, self.graph.N_col_dict[col_case]), dtype=tf.bool))
            setattr(self, "dist_default_"+col_case, 
                    tf.constant(get_dist_default(self.N_sim, self.graph.N_col_dict[col_case]), dtype=tf.float32))
            x_batch, y_batch = get_xy_batch(self.N_sim, self.graph.N_col_dict[col_case])
            setattr(self, "x_batch_"+col_case, 
                    tf.constant(x_batch, dtype=tf.float32))
            setattr(self, "y_batch_"+col_case, 
                    tf.constant(y_batch, dtype=tf.float32))
            
    def apply_col_mask(self):
        for col_case in self.graph.pair_cases:
            setattr(self, "mask_"+col_case, self.graph.col_mask_dict[col_case])
    
    @tf.function
    def pt_pt_dist(self, Tbo_all_res):
        Tbo_all_res = tf.tile(Tbo_all_res, [1, self.N_col_pt_pt, 1,1,1,1])
        vtx1_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx1_pt_pt, axis=-1), axis=-3), [0,1,2], axis=-1)
        vtx2_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx2_pt_pt, axis=-1), axis=-3), [0,1,2], axis=-1)
        dist, flag = distance_pt_pt(vtx1_all, vtx2_all, self.dist1_pt_pt, self.dist2_pt_pt)
        return dist, flag, self.mask_pt_pt

    @tf.function
    def pt_ln_dist(self, Tbo_all_res):
        Tbo_all_res = tf.tile(Tbo_all_res, [1, self.N_col_pt_ln, 1,1,1,1])
        vtx1_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx1_pt_ln, axis=-1), axis=-3), [0,1,2], axis=-1)
        vtx2_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx2_pt_ln, axis=-1), axis=-3), [0,1,2], axis=-1)
        dist, flag = distance_pt_ln(vtx1_all, vtx2_all, self.dist1_pt_ln, self.dist2_pt_ln)
        return dist, flag, self.mask_pt_ln

    @tf.function
    def pt_pl_dist(self, Tbo_all_res):
        Tbo_all_res = tf.tile(Tbo_all_res, [1, self.N_col_pt_pl, 1,1,1,1])
        vtx1_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx1_pt_pl, axis=-1), axis=-3), [0,1,2], axis=-1)
        vtx2_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx2_pt_pl, axis=-1), axis=-3), [0,1,2], axis=-1)
        dist, flag = distance_pt_pl(vtx1_all, vtx2_all, self.dist1_pt_pl, self.dist2_pt_pl, self.N_sim, self.N_col_pt_pl)
        return dist, flag, self.mask_pt_pl

    @tf.function
    def pt_bx_dist(self, Tbo_all_res):
        Tbo_all_res = tf.tile(Tbo_all_res, [1, self.N_col_pt_bx, 1,1,1,1])
        vtx1_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx1_pt_bx, axis=-1), axis=-3), [0,1,2], axis=-1)
        vtx2_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx2_pt_bx, axis=-1), axis=-3), [0,1,2], axis=-1)
        dist, flag = distance_pt_bx(vtx1_all, vtx2_all, self.dist1_pt_bx, self.dist2_pt_bx, self.N_sim, self.N_col_pt_bx)
        return dist, flag, self.mask_pt_bx

    @tf.function
    def ln_ln_dist(self, Tbo_all_res):
        Tbo_all_res = tf.tile(Tbo_all_res, [1, self.N_col_ln_ln, 1,1,1,1])
        vtx1_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx1_ln_ln, axis=-1), axis=-3), [0,1,2], axis=-1)
        vtx2_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx2_ln_ln, axis=-1), axis=-3), [0,1,2], axis=-1)
        dist, flag = distance_ln_ln(vtx1_all, vtx2_all, self.dist1_ln_ln, self.dist2_ln_ln, 
                                    self.N_sim, self.N_col_ln_ln, self.zeros_ln_ln)
        return dist, flag, self.mask_ln_ln

    @tf.function
    def ln_pl_dist(self, Tbo_all_res):
        Tbo_all_res = tf.tile(Tbo_all_res, [1, self.N_col_ln_pl, 1,1,1,1])
        vtx1_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx1_ln_pl, axis=-1), axis=-3), [0,1,2], axis=-1)
        vtx2_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx2_ln_pl, axis=-1), axis=-3), [0,1,2], axis=-1)
        dist, flag = distance_ln_pl(vtx1_all, vtx2_all, self.dist1_ln_pl, self.dist2_ln_pl, 
                                    self.N_sim, self.N_col_ln_pl, self.zeros_ln_pl)
        return dist, flag, self.mask_ln_pl

    @tf.function
    def ln_bx_dist(self, Tbo_all_res):
        Tbo_all_res = tf.tile(Tbo_all_res, [1, self.N_col_ln_bx, 1,1,1,1])
        vtx1_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx1_ln_bx, axis=-1), axis=-3), [0,1,2], axis=-1)
        vtx2_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx2_ln_bx, axis=-1), axis=-3), [0,1,2], axis=-1)
        dist, flag = distance_ln_bx(vtx1_all, vtx2_all, self.dist1_ln_bx, self.dist2_ln_bx, 
                                    self.flag_default_ln_bx, self.dist_default_ln_bx,
                                    self.x_batch_ln_bx, self.y_batch_ln_bx,
                                    4
                                   )
        return dist, flag, self.mask_ln_bx

    @tf.function
    def pl_pl_dist(self, Tbo_all_res):
        Tbo_all_res = tf.tile(Tbo_all_res, [1, self.N_col_pl_pl, 1,1,1,1])
        vtx1_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx1_pl_pl, axis=-1), axis=-3), [0,1,2], axis=-1)
        vtx2_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx2_pl_pl, axis=-1), axis=-3), [0,1,2], axis=-1)
        dist, flag = distance_pl_pl(vtx1_all, vtx2_all, self.dist1_pl_pl, self.dist2_pl_pl, 
                                    self.flag_default_pl_pl, self.dist_default_pl_pl,
                                    self.x_batch_pl_pl, self.y_batch_pl_pl,
                                    4
                                   )
        return dist, flag, self.mask_pl_pl

    @tf.function
    def pl_bx_dist(self, Tbo_all_res):
        Tbo_all_res = tf.tile(Tbo_all_res, [1, self.N_col_pl_bx, 1,1,1,1])
        vtx1_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx1_pl_bx, axis=-1), axis=-3), [0,1,2], axis=-1)
        vtx2_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx2_pl_bx, axis=-1), axis=-3), [0,1,2], axis=-1)
        dist, flag = distance_pl_bx(vtx1_all, vtx2_all, self.dist1_pl_bx, self.dist2_pl_bx, 
                                    self.flag_default_pl_bx, self.dist_default_pl_bx,
                                    self.x_batch_pl_bx, self.y_batch_pl_bx,
                                    4
                                   )
        return dist, flag, self.mask_pl_bx

    @tf.function
    def bx_bx_dist(self, Tbo_all_res):
        Tbo_all_res = tf.tile(Tbo_all_res, [1, self.N_col_bx_bx, 1,1,1,1])
        vtx1_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx1_bx_bx, axis=-1), axis=-3), [0,1,2], axis=-1)
        vtx2_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx2_bx_bx, axis=-1), axis=-3), [0,1,2], axis=-1)
        dist, flag = distance_bx_bx(vtx1_all, vtx2_all, self.dist1_bx_bx, self.dist2_bx_bx, 
                                    self.flag_default_bx_bx, self.dist_default_bx_bx,
                                    self.x_batch_bx_bx, self.y_batch_bx_bx,
                                    4
                                   )
        return dist, flag, self.mask_bx_bx
    
    @tf.function
    def calc_all(self, Tbo_all_res):
        dist_pt_pt, flag_pt_pt, mask_pt_pt = self.pt_pt_dist(Tbo_all_res)
        dist_pt_ln, flag_pt_ln, mask_pt_ln = self.pt_ln_dist(Tbo_all_res)
        dist_pt_pl, flag_pt_pl, mask_pt_pl = self.pt_pl_dist(Tbo_all_res)
        dist_pt_bx, flag_pt_bx, mask_pt_bx = self.pt_bx_dist(Tbo_all_res)
        dist_ln_ln, flag_ln_ln, mask_ln_ln = self.ln_ln_dist(Tbo_all_res)
        dist_ln_pl, flag_ln_pl, mask_ln_pl = self.ln_pl_dist(Tbo_all_res)
        dist_ln_bx, flag_ln_bx, mask_ln_bx = self.ln_bx_dist(Tbo_all_res)
        dist_pl_pl, flag_pl_pl, mask_pl_pl = self.pl_pl_dist(Tbo_all_res)
        dist_pl_bx, flag_pl_bx, mask_pl_bx = self.pl_bx_dist(Tbo_all_res)
        dist_bx_bx, flag_bx_bx, mask_bx_bx = self.bx_bx_dist(Tbo_all_res)
        return (
            dist_pt_pt, flag_pt_pt, mask_pt_pt,
            dist_pt_ln, flag_pt_ln, mask_pt_ln,
            dist_pt_pl, flag_pt_pl, mask_pt_pl,
            dist_pt_bx, flag_pt_bx, mask_pt_bx,
            dist_ln_ln, flag_ln_ln, mask_ln_ln,
            dist_ln_pl, flag_ln_pl, mask_ln_pl,
            dist_ln_bx, flag_ln_bx, mask_ln_bx,
            dist_pl_pl, flag_pl_pl, mask_pl_pl,
            dist_pl_bx, flag_pl_bx, mask_pl_bx,
            dist_bx_bx, flag_bx_bx, mask_bx_bx)

# Graph

In [10]:
from collections import defaultdict

In [11]:
class GraphModel(tf.keras.Model):
    def __init__(self, robot_info, gitem_list, urdf_content, N_sim, 
                 alpha_jc=5, alpha_fc=200, alpha_jl=1, alpha_cl=1,alpha_cs=0, 
                 LIM=np.pi, LIM_BOUND=1e-1, COL_BOUND=1e-2, learning_rate=5e-3, col_iteration = 20):
        super(GraphModel, self).__init__()
        self.alpha_jc = alpha_jc
        self.alpha_fc = alpha_fc
        self.alpha_jl = alpha_jl
        self.alpha_cl = alpha_cl
        self.alpha_cs = alpha_cs
        self.N_sim = N_sim
        self.LIM = LIM
        self.LIM_BOUND = LIM_BOUND
        self.COL_BOUND = COL_BOUND
        self.learning_rate = learning_rate
        self.col_iteration = col_iteration
        self.robot_info = robot_info
        self.robot = RobotLayer(
            robot_info.link_info_list, rname = robot_info.rname, dim=N_sim)
        self.joint_constraint = JointConstraintLoss(self.robot)
        self.frame_constraint = FrameConstraintLoss()
        self.robot_base = [robot_info.base_frame]*N_sim
        self.set_custom_loss(lambda *args: tf.constant(0.0), 0)
        self.link_name_list = self.robot.link_name_list
        self.link_idx_dict = {name: idx for name, idx in zip(self.link_name_list, range(len(self.link_name_list)))}
        self.adjacency_list = [
            list(map(
                lambda x: self.link_idx_dict[x],
                [lname for lname in get_adjacent_links(link_name, urdf_content) if lname in self.link_name_list]
            )) for link_name in self.link_name_list
        ]
        link_num = len(self.adjacency_list)
        self.adjacency_mat = np.zeros((link_num,link_num), dtype=np.bool)
        for i_adj, adj in zip(range(link_num), self.adjacency_list):
            self.adjacency_mat[i_adj, adj] = True
            
        self.object_dict = {}
        self.object_name_list = []
        self.object_vertice_list = []
        self.object_link_idx_list = []
        self.object_collision_flags = []
        for gitem in gitem_list:
            self.object_dict[gitem.name] = ObjectLayer(gitem, N_sim)
            self.object_collision_flags += [gitem.collision]
            self.object_name_list += [gitem.name]
            self.object_vertice_list += [np.pad(gitem.get_vertice(),((0,0),(0,1)),'constant', constant_values=(1))]
            self.object_link_idx_list += [0]
        self.num_object = len(self.object_name_list)
#         self.object_vertice_mat = tf.constant(self.object_vertice_list, dtype="float32")
        self.build_collision_combinations()
        self.set_collision_vtx()
        self.optimizer = tf.optimizers.SGD(learning_rate=learning_rate)

    def build_collision_combinations(self):
        obj_idx_list = np.arange(len(self.object_name_list))
        self.col_combinations = list(combinations(obj_idx_list,2))

        for i_c in range(len(self.col_combinations)):
            i_c_back = len(self.col_combinations)-1-i_c
            cmb = self.col_combinations[i_c_back]
            if not(self.object_collision_flags[cmb[0]] 
                   and self.object_collision_flags[cmb[1]]):
                del self.col_combinations[i_c_back]

        self.pair_cases = ['pt_pt', 'pt_ln', 'pt_pl', 'pt_bx',
                           'ln_ln', 'ln_pl', 'ln_bx',
                           'pl_pl', 'pl_bx','bx_bx']
        self.col_pair_dict = defaultdict(lambda:[])
        for comb in self.col_combinations:
            obj1 = self.object_dict[self.object_name_list[comb[0]]]
            obj2 = self.object_dict[self.object_name_list[comb[1]]]
            if obj1.gitem.gtype == GeoType.SPHERE:
                if obj2.gitem.gtype == GeoType.SPHERE:
                    self.col_pair_dict['pt_pt'] += [comb]
                elif obj2.gitem.gtype == GeoType.CYLINDER or obj2.gitem.gtype == GeoType.LINE:
                    self.col_pair_dict['pt_ln'] += [comb]
                elif obj2.gitem.gtype == GeoType.PLANE:
                    self.col_pair_dict['pt_pl'] += [comb]
                elif obj2.gitem.gtype == GeoType.BOX:
                    self.col_pair_dict['pt_bx'] += [comb]
            elif obj1.gitem.gtype == GeoType.CYLINDER or obj2.gitem.gtype == GeoType.LINE:
                if obj2.gitem.gtype == GeoType.SPHERE:
                    self.col_pair_dict['pt_ln'] += [(comb[1],comb[0])]
                elif obj2.gitem.gtype == GeoType.CYLINDER or obj2.gitem.gtype == GeoType.LINE:
                    self.col_pair_dict['ln_ln'] += [comb]
                elif obj2.gitem.gtype == GeoType.PLANE:
                    self.col_pair_dict['ln_pl'] += [comb]
                elif obj2.gitem.gtype == GeoType.BOX:
                    self.col_pair_dict['ln_bx'] += [comb]
            elif obj1.gitem.gtype == GeoType.PLANE:
                if obj2.gitem.gtype == GeoType.SPHERE:
                    self.col_pair_dict['pt_pl'] += [(comb[1],comb[0])]
                elif obj2.gitem.gtype == GeoType.CYLINDER or obj2.gitem.gtype == GeoType.LINE:
                    self.col_pair_dict['ln_pl'] += [(comb[1],comb[0])]
                elif obj2.gitem.gtype == GeoType.PLANE:
                    self.col_pair_dict['pl_pl'] += [comb]
                elif obj2.gitem.gtype == GeoType.BOX:
                    self.col_pair_dict['pl_bx'] += [comb]
            elif obj1.gitem.gtype == GeoType.BOX:
                if obj2.gitem.gtype == GeoType.SPHERE:
                    self.col_pair_dict['pt_bx'] += [(comb[1],comb[0])]
                elif obj2.gitem.gtype == GeoType.CYLINDER or obj2.gitem.gtype == GeoType.LINE:
                    self.col_pair_dict['ln_bx'] += [(comb[1],comb[0])]
                elif obj2.gitem.gtype == GeoType.PLANE:
                    self.col_pair_dict['pl_bx'] += [(comb[1],comb[0])]
                elif obj2.gitem.gtype == GeoType.BOX:
                    self.col_pair_dict['bx_bx'] += [comb]
            
    def assign_frame_dict(self, gframeset_list):
        frame_dict = {k: [] for k in self.object_dict.keys()}
        link_dict = {k: [] for k in self.object_dict.keys()}
        for gframeset in gframeset_list:
            for k, gframe in gframeset.items():
                frame_dict[k] += [gframe.Toff]
                link_dict[k] += [self.robot.link_name_list.index(gframe.link_name)]
        for k in frame_dict.keys():
            self.object_dict[k].set_frame(np.array(frame_dict[k]), np.array(link_dict[k]), self.robot.len_Q)
            i_obj = self.get_object_index(k)
            self.object_link_idx_list[i_obj] = np.array(link_dict[k])
        self.object_link_idx_mat = np.transpose(self.object_link_idx_list)
        self.apply_col_mask()
            
    def get_object_index(self, name):
        return graph.object_name_list.index(name)
            
    @tf.function
    def assign_Q(self, Q):
        self.robot.assign_Q(Q)
            
    @tf.function
    def get_Q(self):
        return self.robot.get_Q()

    @tf.function
    def call(self, inputs=None):
        T_all = self.robot(self.robot_base)
        Tbo_all = []
        for obj_name in self.object_name_list:
            Tbo_all += [self.object_dict[obj_name](T_all)] #(Nobj,N_sim,4,4)
        Tbo_all = K.stack(Tbo_all, axis=1) #(N_sim,Nobj,4,4)
        return T_all, Tbo_all
    
    @tf.function
    def calc_joint_limit(self):
        Q = self.get_Q()
        return K.sum(1/((K.min(((self.LIM-Q), (self.LIM+Q)), axis=-1)/self.LIM_BOUND)**3))
    
    @tf.function
    def calc_loss(self, T_all, Tbo_all, Qtar, binQ, Ttar, binT):
        jl_loss = self.calc_joint_limit()
        jc_loss = self.joint_constraint((Qtar, binQ))
        fc_loss = self.frame_constraint((T_all[:,-1,:,:],Ttar, binT))
        cl_loss = self.calc_collision_loss(T_all, Tbo_all)
        return self.alpha_jc*jc_loss + self.alpha_fc*fc_loss + \
                self.alpha_jl*jl_loss + self.alpha_cs*self.calc_custom_loss(T_all, Tbo_all) \
                + self.alpha_cl*cl_loss 
                
    
    @tf.function
    def forward(self, Qtar, binQ, Ttar, binT):
        T_all, Tbo_all= graph(None)
        loss = self.calc_loss(T_all, Tbo_all, Qtar, binQ, Ttar, binT)
        return loss
    
    @tf.function
    def update_once(self, Qtar, binQ, Ttar, binT, max_gradient):
        with tf.GradientTape() as g:
            # Forward pass.
            loss = self.forward(Qtar, binQ, Ttar, binT)

        # Variables to update, i.e. trainable variables.
        trainable_variables = self.trainable_variables

        # Compute gradients.
        gradients = g.gradient(loss, trainable_variables)
        gradients = tf.unstack(clip_gradient(gradients, max_gradient))
    
        # Update W and b following gradients.
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return loss
    
    def set_custom_loss(self, custom_loss, alpha_cs, *args, **kwargs):
        self.custom_loss = custom_loss
        self.alpha_cs = alpha_cs
        self.cl_args = args
        self.cl_kwargs = kwargs
        
    def calc_custom_loss(self, T_all, Tbo_all):
        return self.custom_loss(self, T_all, Tbo_all, *self.cl_args, **self.cl_kwargs)

    @tf.function
    def test_collision(self, T_all, Tbo_all):
        Pbo_all = tf.gather(tf.gather(Tbo_all, [0,1,2], axis=-2), 3, axis=-1)
        col_combs_masked_batch = self.get_collision_combination()
        batch_flat = tf.repeat(np.reshape(np.arange(self.N_sim), (self.N_sim, 1)), (self.N_col,)*self.N_sim,axis=0)
        col_combs_masked_flat = tf.concat([batch_flat, col_combs_masked_batch], axis=-1)
        vtx_tf = tf.gather(
            K.sum(tf.expand_dims(Tbo_all, axis=-3)*tf.expand_dims(self.object_vertice_mat, axis=-2), axis=-1), 
            [0,1,2], axis=-1)
        vtx_tf_1 = tf.reshape(tf.gather_nd(vtx_tf, tf.gather(col_combs_masked_flat, [0,1], axis=1)), (self.N_sim, self.N_col, -1,3))
        vtx_tf_2 = tf.reshape(tf.gather_nd(vtx_tf, tf.gather(col_combs_masked_flat, [0,2], axis=1)), (self.N_sim, self.N_col, -1,3))
        Pbo_all_1 = tf.reshape(tf.gather_nd(Pbo_all, tf.gather(col_combs_masked_flat, [0,1], axis=1)), (self.N_sim, self.N_col,3))
        Pbo_all_2 = tf.reshape(tf.gather_nd(Pbo_all, tf.gather(col_combs_masked_flat, [0,2], axis=1)), (self.N_sim, self.N_col,3))
        v_batch = tf.expand_dims(Pbo_all_1-Pbo_all_2, axis=-2)
        v_batch = tf.stop_gradient(tf.linalg.cross(v_batch, self.x_batch)+tf.linalg.cross(v_batch, self.y_batch))
        dist, flag = test_collision_batch(vtx_tf_1, vtx_tf_2, 
                             v_batch, self.flag_default, self.dist_default, IterationAllowed=self.col_iteration)
        return dist, flag, col_combs_masked_flat 
    
    @tf.function
    def calc_collision_loss(self, T_all, Tbo_all):
        dist, flag, _  = self.test_collision(T_all, Tbo_all)
        return K.sum(1/((dist/self.COL_BOUND)**3))
    
    def get_collision_combination_names(self):
        comb_names = []
        for comb in self.get_collision_combination():
            comb_names += ["{} - {}".format(self.object_name_list[comb[0]], graph.object_name_list[comb[1]])]
            
    def get_vtx_pair(self, pair_list, N_vtx1, N_vtx2):
        N_col = len(pair_list)
        pair_vtx1 = np.zeros((1, len(pair_list), len(self.object_name_list), N_vtx1, 1, 4)) # (N_sim, N_col, N_obj, N_vtx, 1, 4)
        pair_vtx2 = np.zeros((1, len(pair_list), len(self.object_name_list), N_vtx2, 1, 4))
        pair_dist1 = np.zeros((1, len(pair_list), 1))
        pair_dist2 = np.zeros((1, len(pair_list), 1))
        for i_c, comb in zip(range(len(pair_list)), pair_list):
            pair_vtx1[0,i_c,comb[0],:,0] = np.pad(self.object_dict[self.object_name_list[comb[0]]].gitem.get_vertice(),((0,0),(0,1)),'constant', constant_values=(1))
            pair_vtx2[0,i_c,comb[1],:,0] = np.pad(self.object_dict[self.object_name_list[comb[1]]].gitem.get_vertice(),((0,0),(0,1)),'constant', constant_values=(1))
            pair_dist1[0,i_c, 0] = self.object_dict[self.object_name_list[comb[0]]].gitem.get_radius()
            pair_dist2[0,i_c, 0] = self.object_dict[self.object_name_list[comb[1]]].gitem.get_radius()
        pair_vtx1 = tf.constant(pair_vtx1, dtype=tf.float32)
        pair_vtx2 = tf.constant(pair_vtx2, dtype=tf.float32)
        pair_dist1 = tf.constant(pair_dist1, dtype=tf.float32)
        pair_dist2 = tf.constant(pair_dist2, dtype=tf.float32)
        return pair_vtx1, pair_vtx2, pair_dist1, pair_dist2, N_col
    
    def set_collision_vtx(self):
        self.col_vtx1_dict = {}
        self.col_vtx2_dict = {}
        self.col_dist1_dict = {}
        self.col_dist2_dict = {}
        self.N_col_dict = {}
        self.N_vtx_dict = {"pt":1, "ln":2, "pl":4, "bx":8}
        for col_case in self.pair_cases:
            self.col_vtx1_dict[col_case], self.col_vtx2_dict[col_case], \
                self.col_dist1_dict[col_case], self.col_dist2_dict[col_case], self.N_col_dict[col_case] = \
                    self.get_vtx_pair(
                        self.col_pair_dict[col_case], self.N_vtx_dict[col_case[:2]], self.N_vtx_dict[col_case[3:]])
        self.dcal = DistanceCalculator(self)
        
    def apply_col_mask(self):
        self.col_mask_dict = {}
        for col_case in self.pair_cases:
            N_col = self.N_col_dict[col_case]
            mask = np.zeros((self.N_sim, N_col, 1), dtype=np.float32)
            for i_col, comb in zip(range(N_col), self.col_pair_dict[col_case]):
                mask[:, i_col, 0] = tf.gather_nd(self.adjacency_mat, self.object_link_idx_mat[:, comb])
            self.col_mask_dict[col_case] = 1-mask
        self.dcal.apply_col_mask()
        

In [12]:
N_sim = 50
N_joints = 9
DOF = 6
gtimer = GlobalTimer()


gtimer.tic("initialize")
robot_info = RobotInfo(link_info_list, rname = "rbt1", base_frame=np.identity(4,dtype=np.float32))
graph = GraphModel(robot_info=robot_info, gitem_list=gitem_list, urdf_content=urdf_content, 
                   N_sim=N_sim, learning_rate=5e-3, 
                   alpha_cl=0.001)
Q_ = np.array([(0, 0,)+tuple(ZERO_JOINT_POSE+(np.random.rand(DOF)*2-1)*np.pi/100)+(0,) for _ in range(N_sim)], dtype=np.float32)
Ttar_ = SE3(Rot_zyx(0,0,np.pi),(0.5,0,0.00)).astype(np.float32)
gframe_dict_list = [gframe_dict]*N_sim
graph.assign_Q(Q_)
graph.assign_frame_dict(gframe_dict_list)
T_all, Tbo_all = graph(None)
Tbo_all_res = tf.reshape(Tbo_all, (graph.N_sim, 1, graph.num_object, 1, 4,4))
res = graph.dcal.calc_all(Tbo_all_res)
gtimer.toc("initialize")

In [13]:
gtimer.reset()
gtimer.tic("assign")
graph.assign_Q(Q_)
graph.assign_frame_dict(gframe_dict_list)
gtimer.toc("assign")
print(gtimer)

assign: 	6.0 ms/1 = 5.556 ms 



In [14]:
Q_ = np.array([(0, 0,)+tuple(ZERO_JOINT_POSE+(np.random.rand(DOF)*2-1)*np.pi/100)+(0,) for _ in range(N_sim)], dtype=np.float32)
Ttar_ = SE3(Rot_zyx(0,0,np.pi),(0.5,0,0.00)).astype(np.float32)

gtimer.reset()
res_vec = []
for _ in range(50):
    gtimer.tic("calc")
    T_all, Tbo_all = graph(None)
    Tbo_all_res = tf.reshape(Tbo_all, (graph.N_sim, 1, graph.num_object, 1, 4,4))
    gtimer.toc("calc")
    gtimer.tic("collision")
    res = graph.dcal.calc_all(Tbo_all_res)
    gtimer.toc("collision")
    res_vec += [res]
print(gtimer)

calc: 	101.0 ms/50 = 2.018 ms 
collision: 	326.0 ms/50 = 6.519 ms 



In [ ]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as mplot3d

fig = plt.figure(figsize=(10, 10))
sub = fig.add_subplot(1,1,1,projection="3d")
# x, y, z = np.pad(np.transpose(pl2[0,0]), [[0,0],[1,0]])
x,y,z = np.transpose(vtx1_all[0,10])
sub.plot(x,y,z,'-d')
x,y,z = np.transpose(vtx2_all[0,10])
sub.plot(x,y,z,'-d')
sub.plot([0],[0],[0],'+')
sub.set_xlabel('x')
sub.set_ylabel('y')
sub.set_zlabel('z')

# sub.view_init(elev=0., azim=0)

In [ ]:
q = Q_[0]
gframevec = [gframe_dict[gitem.name] for gitem in gitem_list]
pose_list = [q[2:-1]]
gframevec_list = [gframevec]
show_motion(pose_list, marker_list, gframevec_list, pub, joints, error_skip=1e-6, period=1e-6)

In [ ]:

dist, flag = distance_pt_pl(zeros_pt, pl2, dist1, dist2, N_sim, N_col)
dist = dist - (dist1+dist2)
flag = tf.less_equal(dist, 0)

# SGD Optimizer

In [ ]:
Q_ = np.array([(0, 0,)+tuple(ZERO_JOINT_POSE+(np.random.rand(DOF)*2-1)*np.pi/100)+(0,) for _ in range(N_sim)], dtype=np.float32)
Ttar_ = SE3(Rot_zyx(0,0,np.pi),(0.5,0,0.00)).astype(np.float32)
gframe_dict_list = [gframe_dict]*N_sim
graph.assign_Q(Q_)
graph.assign_frame_dict(gframe_dict_list)

In [ ]:
time_vec = []
Q_list = [list(map(lambda x: x.numpy(), graph.get_Q()))]
# Tbo_all_list = [np.array([[gframe_dict[gitem.name].Toff for gitem in gitem_list]]*N_sim)]
N_iter = 100
# Run training for the given number of steps.
Qtar, binQ, Ttar, binT = (np.zeros((N_sim, N_joints), dtype='float32'), np.zeros(N_sim, dtype='float32'),
                      np.array([Ttar_]*N_sim), np.ones(N_sim, dtype='float32'))
dist_list = []
loss_list = []
jl_loss_list = []
jc_loss_list = []
fc_loss_list = []
cl_loss_list = []
gradients_list = []
gtimer.reset()
for _ in range(N_iter):
    # Run the optimization to update W and b values.
    T_all, Tbo_all = graph(None)
    dist_list += [K.min(graph.test_collision(T_all, Tbo_all)[0]).numpy()]
    jl_loss = graph.calc_joint_limit()
    jc_loss = graph.joint_constraint((Qtar, binQ))
    fc_loss = graph.frame_constraint((T_all[:,-1,:,:],Ttar, binT))
    cl_loss = graph.calc_collision_loss(T_all, Tbo_all)
    gtimer.tic("update")
    loss = graph.update_once(Qtar, binQ, Ttar, binT, max_gradient=10)
#     max_gradient=10

#     with tf.GradientTape() as g:
#         # Forward pass.
#         loss = graph.forward(Qtar, binQ, Ttar, binT)

#     # Variables to update, i.e. trainable variables.
#     trainable_variables = graph.trainable_variables

#     # Compute gradients.
#     gradients = g.gradient(loss, trainable_variables)
#     gradients = tf.unstack(clip_gradient(gradients, max_gradient))

#     # Update W and b following gradients.
#     graph.optimizer.apply_gradients(zip(gradients, graph.trainable_variables))
    gtimer.toc("update")
    gtimer.tic("record")
    Q_list += [graph.get_Q().numpy()]
    loss_list += [loss]
#     gradients_list += [gradients]
    jl_loss_list += [jl_loss]
    jc_loss_list += [jc_loss]
    fc_loss_list += [fc_loss]
    cl_loss_list += [cl_loss]
    gtimer.toc("record")
    if isnan(dist_list[-1]) or dist_list[-1]<0:
        break
gtimer.print_time_log()
print("loss: {}".format(loss))

In [ ]:
plt.plot(dist_list)

In [ ]:
for i_sim in range(1): # N_sim):
    for i_iter in range(0,len(Q_list)):
        q = Q_list[i_iter][i_sim]
        gframevec = [gframe_dict[gitem.name] for gitem in gitem_list]
        pose_list = [q[2:-1]]
        gframevec_list = [gframevec]
        show_motion(pose_list, marker_list, gframevec_list, pub, joints, error_skip=1e-6, period=1e-6)
        time.sleep(10e-2)

# Testing

In [ ]:
@tf.function
def _loop_pickTetrahedron_batch(a, b, c, d, v, dist, flag, FX1_batch, FX2_batch, iter):
    a,b,c,v_,dist_ = pickClosestFace_batch(a,b,c,d)
    v_ = tf.stop_gradient(v_)
    a_,b_,c_,d_ = nearest_simplex4_batch(a,b,c,v_, dist_, FX1_batch,FX2_batch)
    flag = tf.less_equal(dist_,0)
    def_case = tf.stop_gradient(tf.cast(flag, tf.float32))
    def_case_not = tf.stop_gradient(tf.cast(tf.logical_not(flag), tf.float32))
    def_case_exp = tf.expand_dims(def_case,axis=-1)
    def_case_not_exp = tf.expand_dims(def_case_not,axis=-1)
    d = def_case_not*d_ + def_case*d
    c = def_case_not*c_ + def_case*c
    b = def_case_not*b_ + def_case*b
    a = def_case_not*a_ + def_case*a
    v = def_case_not_exp*v_ + def_case_exp*v
    dist = def_case_not*dist_ + def_case*dist
#     d = d_
#     c = c_
#     b = b_
#     a = a_
#     v = v_
#     dist = dist_
    iter += 1
    return a, b, c, d, v, dist, flag, FX1_batch, FX2_batch, iter

@tf.function
def _cond_pickTetrahedron_batch(a, b, c, d, v, dist, flag, FX1_batch, FX2_batch, iter):
    return tf.reduce_any(tf.logical_not(flag))

In [ ]:
# Q_ = np.array([(0, 0,)+tuple(ZERO_JOINT_POSE+(np.random.rand(DOF)*2-1)*np.pi/100)+(0,) for _ in range(N_sim)], dtype=np.float32)
Q_ = np.array([[ 0.        ,  0.        ,  0.00195152, -1.8906006 ,  1.90172   , -0.01040376, -0.00901779,  0.02714482,  0.]
               for _ in range(N_sim)], dtype=np.float32)

graph.assign_Q(Q_)
graph.assign_frame_dict(gframe_dict_list)
Ttar_ = SE3(Rot_zyx(0,0,np.pi),(0.5,0,-0.05)).astype(np.float32)
gframe_dict_list = [gframe_dict]*N_sim
graph.assign_Q(Q_)
graph.assign_frame_dict(gframe_dict_list)

In [ ]:
T_all, Tbo_all = graph(None)
self = graph
Pbo_all = tf.gather(tf.gather(Tbo_all, [0,1,2], axis=-2), 3, axis=-1)
col_combs_masked_batch = self.get_collision_combination()
batch_flat = tf.repeat(np.reshape(np.arange(self.N_sim), (self.N_sim, 1)), (self.N_col,)*self.N_sim,axis=0)
col_combs_masked_flat = tf.concat([batch_flat, col_combs_masked_batch], axis=-1)
vtx_tf = tf.gather(
    K.sum(tf.expand_dims(Tbo_all, axis=-3)*tf.expand_dims(self.object_vertice_mat, axis=-2), axis=-1), 
    [0,1,2], axis=-1)
vtx_tf_1 = tf.reshape(tf.gather_nd(vtx_tf, tf.gather(col_combs_masked_flat, [0,1], axis=1)), (self.N_sim, self.N_col, -1,3))
vtx_tf_2 = tf.reshape(tf.gather_nd(vtx_tf, tf.gather(col_combs_masked_flat, [0,2], axis=1)), (self.N_sim, self.N_col, -1,3))
Pbo_all_1 = tf.reshape(tf.gather_nd(Pbo_all, tf.gather(col_combs_masked_flat, [0,1], axis=1)), (self.N_sim, self.N_col,3))
Pbo_all_2 = tf.reshape(tf.gather_nd(Pbo_all, tf.gather(col_combs_masked_flat, [0,2], axis=1)), (self.N_sim, self.N_col,3))
v_batch = tf.expand_dims(Pbo_all_1-Pbo_all_2, axis=-2)
v_batch = tf.linalg.cross(v_batch, self.x_batch)+tf.linalg.cross(v_batch, self.y_batch)
# dist, flag = test_collision_batch(vtx_tf_1, vtx_tf_2, 
#                      v_batch, self.flag_default, self.dist_default)

In [ ]:
FX1_batch, FX2_batch, v_batch,flag_default,dist_default, IterationAllowed = \
    vtx_tf_1, vtx_tf_2, v_batch, self.flag_default, self.dist_default, 10
a, b = pickLineTF_batch(v_batch, FX2_batch, FX1_batch)
a, b, c, flag = PickTriangleTF_batch(a,b,FX2_batch,FX1_batch,flag_default,IterationAllowed)
# a,b,c,d,dist,flag, iter = pickTetrahedronTF_batch(a,b,c,FX2_batch,FX1_batch,flag_default,dist_default,IterationAllowed)

a,b,c,FX1_batch,FX2_batch,flag_default,dist_default,IterationAllowed = \
    a,b,c,FX2_batch,FX1_batch,flag_default,dist_default,IterationAllowed
    ab = b-a
    ac = c-a

    # Normal to face of triangle
    abc = tf.linalg.cross(ab,ac)
    v, abc_nm = tf.linalg.normalize(abc,axis=-1)
    v = tf.stop_gradient(v)
    dist = K.sum(v*a,axis=-1, keepdims=True)
    v = tf.expand_dims(v, axis=-2)
    v = tf.stop_gradient(v)

    a,b,c,d = nearest_simplex4_batch(a,b,c,v,dist, FX1_batch,FX2_batch)


    a, b, c, d, v, dist, flag, _, _, iter = tf.while_loop(
        _cond_pickTetrahedron_batch, _loop_pickTetrahedron_batch, 
        (a,b,c,d,v,dist_default,flag_default, FX1_batch, FX2_batch, 0), 
        parallel_iterations=10, maximum_iterations=IterationAllowed
    )
    v = tf.stop_gradient(v)
    v_, dist_ = direct(b,c,d,v,dist)
    v_ = tf.stop_gradient(v_)
    # v_ = tf.sign(tf.expand_dims(dist,axis=-1))*v_
    # dist2 = tf.sign(dist)*dist_
    a = support_batch(FX2_batch,FX1_batch,v_) # Tetrahedron new point
    v_rs = tf.reduce_sum(v, axis=-2)
    dist3_ = K.sum(-a*v_rs, axis=-1)
    dotted = tf.reduce_sum(K.sum(v_*v, axis=-1), axis=-1)
    dist = dist3_/dotted

In [ ]:
K.sum(-a*tf.reduce_sum(v, axis=-2), axis=-1)[0]

In [ ]:
dist[0,:,0]

In [ ]:
dist[0,14]

In [ ]:
K.sum(-a[0,14]*v[0,14])

In [ ]:
K.sum(-b[0,14]*v[0,14])

In [ ]:
K.sum(-c[0,14]*v[0,14])

In [ ]:
a_bak = a

In [ ]:
plt.plot(a[0]-a_bak[0])
plt.ylim([-1e-1, 1e-1])

In [ ]:
dd = (tf.reduce_sum(dist2,axis=-1)-dist3)[0]
plt.plot(dd)
# plt.plot(dotted[0])

In [ ]:
i_max_dd = K.argmax(dd)
i_max_dd

In [ ]:
dist[0, i_max_dd] #normal

In [ ]:
dist_[0, i_max_dd] # refined

In [ ]:
dist2[0, i_max_dd] # resigned

In [ ]:
dist[0, i_max_dd]/dotted[0, i_max_dd] # original rescale

In [ ]:
dist3_[0, i_max_dd] # new a normal

In [ ]:
dist3[0, i_max_dd]

In [ ]:
v[0, i_max_dd] # original

In [ ]:
v_[0, i_max_dd] # refined

In [ ]:
i_max_dd = 0 

In [ ]:
comb = graph.get_collision_combination()[i_max_dd]
print(graph.object_name_list[comb[0]])
print(graph.object_name_list[comb[1]])

In [ ]:
offset = np.mean(FX1_batch[0][i_max_dd],axis=-2)

In [ ]:

import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as mplot3d

fig = plt.figure(figsize=(10, 10))

sub = fig.add_subplot(1,1,1,projection="3d")
x, y, z = np.transpose(FX1_batch[0][i_max_dd])
sub.plot(x,y,z,'.')
x, y, z = np.transpose(FX2_batch[0][i_max_dd])
sub.plot(x,y,z,'.')
x, y, z = np.transpose(offset + np.pad(v_[0,i_max_dd]/10, ((1,0),(0,0))))
sub.plot(x,y,z,'-o')
x, y, z = np.transpose(offset + np.pad(v[0,i_max_dd]/10, ((1,0),(0,0))))
sub.plot(x,y,z,'-o')

In [ ]:
v[0,14]

In [ ]:
v_[0,14]

In [ ]:
K.argmin(dist[0,:,0])

In [ ]:
K.min(dist2)

In [ ]:
K.min(dist)

In [ ]:
graph.get_collision_combination()[14]

In [ ]:
graph.object_name_list[3]

In [ ]:
graph.object_name_list[5]

In [ ]:
q = Q_[0]
gframevec = [gframe_dict[gitem.name] for gitem in gitem_list]
pose_list = [q[2:-1]]
gframevec_list = [gframevec]
show_motion(pose_list, marker_list, gframevec_list, pub, joints, error_skip=1e-6, period=1e-6)

In [ ]:
plt.plot(loss_list, label="total")
print(len(loss_list))

In [ ]:
plt.plot(loss_list, label="total")
plt.plot(np.multiply(jl_loss_list, graph.alpha_jl), label="joint limit")
plt.plot(np.multiply(jc_loss_list, graph.alpha_jc), label="joint target")
plt.plot(np.multiply(fc_loss_list, graph.alpha_fc), label="frame target")
plt.plot(np.multiply(cl_loss_list, graph.alpha_cl), label="collision dist")
plt.legend()
axes = plt.gca()
# axes.set_ylim([-1e1,5e1])
# axes.set_xlim([0,1e1])

In [ ]:
plt.plot(dist_list)

In [ ]:
for i_sim in range(1): # N_sim):
    for i_iter in range(0,len(Q_list)):
        q = Q_list[i_iter][i_sim]
        gframevec = [gframe_dict[gitem.name] for gitem in gitem_list]
        pose_list = [q[2:-1]]
        gframevec_list = [gframevec]
        show_motion(pose_list, marker_list, gframevec_list, pub, joints, error_skip=1e-6, period=1e-6)
        time.sleep(10e-2)

In [ ]:
Q_all = np.array(Q_list)
Q_all_i = np.array([Q_all[i_iter, i_sim,:] for i_iter in range(len(Q_list))])
plt.plot(Q_all_i)

In [ ]:
plt.plot(Q_all_i[1:,:]-Q_all_i[:-1,:])

# Collision test

In [ ]:
Q_ = np.array([(0, 0,)+tuple(ZERO_JOINT_POSE+(np.random.rand(DOF)*2-1)*np.pi/100)+(0,) for _ in range(N_sim)], dtype=np.float32)
graph.assign_Q(Q_)
graph.assign_frame_dict(gframe_dict_list)

In [ ]:
Ttar_ = SE3(Rot_zyx(0,0,np.pi),(0.5,0,-0.05)).astype(np.float32)
gframe_dict_list = [gframe_dict]*N_sim
graph.assign_Q(Q_)
graph.assign_frame_dict(gframe_dict_list)
T_all, Tbo_all = graph(None)
show_motion([graph.get_Q().numpy()[0][2:-1]], marker_list, [[gframe_dict[gitem.name] for gitem in gitem_list]], pub, joints, error_skip=1e-6, period=1e-6)

In [ ]:
T_all, Tbo_all = graph(None)
gtimer.reset()
for _ in range(10):
    gtimer.tic("ctest")
    dist, flag, mask = graph.test_collision(T_all, Tbo_all)
    gtimer.toc("ctest")
print(gtimer)
# dist[0]
# gtimer.reset()
# for _ in range(100):
#     gtimer.tic("col")
#     dist, flag = collision_loss_test(graph, T_all, Tbo_all, flag_default, dist_default)
#     gtimer.toc("col")
# gtimer.print_time_log()

In [ ]:
graph.calc_collision_loss(T_all, Tbo_all)

In [ ]:
plt.figure(figsize=(10,8))
plt.plot([0, N_col],[0,0],'-ob')
plt.grid()
dist0 = dist[0]
mask0 = mask[:N_col]
for i_m in range(len(mask0)):
    plt.plot([i_m, i_m],mask0[i_m,1:],'-o')
plt.plot(dist0*10,'-ok',linewidth=2)

In [ ]:
graph.object_name_list

# Collision error case 

In [ ]:
Q_error = np.loadtxt("Q_error.csv",delimiter=",",dtype=np.float32)
graph.assign_Q(Q_error)
graph.assign_frame_dict(gframe_dict_list)

T_all, Tbo_all = graph(None)
dist = K.min(graph.test_collision(T_all, Tbo_all)[0]).numpy()
cl_loss = graph.calc_collision_loss(T_all, Tbo_all)
print("dist: {}".format(dist))

In [ ]:
q = Q_error[0]
gframevec = [gframe_dict[gitem.name] for gitem in gitem_list]
pose_list = [q[2:-1]]
gframevec_list = [gframevec]
show_motion(pose_list, marker_list, gframevec_list, pub, joints, error_skip=1e-6, period=1e-6)

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import mpl_toolkits.mplot3d as mplot3d

In [ ]:
fig = plt.figure()
sub = fig.add_subplot(1,1,1,projection="3d")
sub.plot(x,y,z)
plt.show()

In [ ]:
t = np.linspace(0,4*np.pi,500)
x = np.sin(t)
x = np.cos(t)
y = np.sin(t)
z = t